In [49]:
from json import dumps, loads
import yaml
from pathlib import Path
from fhir.resources import construct_fhir_element
from fhir.resources.observation import Observation as Obs
from fhir.resources.questionnaireresponse import QuestionnaireResponse as QR
from fhir.resources.codesystem import CodeSystem as CS
from survey_obs import survey_obs

In [50]:
my_profile = "http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-screening-assessment"
category = 'disability-status'
category_path = Path(f'/Users/ehaas/Documents/FHIR/US-Core/input/resources/CodeSystem-us-core-category.json')
survey_name = 'PHQ9' #'AHC-HRSN' #'PRAPARE' #'HVS' # 
survey_code =  '44249-1' #'96777-8' #'88121-9'  # don't need if already grouped like PRAPARE
survey_code_name ='PHQ-9 quick depression assessment panel [Reported.PHQ]' #'Accountable health communities (AHC) health-related social needs screening (HRSN) tool' #Hunger Vital Sign [HVS]' # don't need if already grouped like PRAPARE
create_survey_panel = True  #|False if already grouped in QR like PRAPARE
survey_path = r'/Users/ehaas/Documents/FHIR/Healthedata1-Sandbox/input/examples-yaml/QuestionnaireResponse-5774968.yml'
#survey_path = r'/Users/ehaas/Documents/FHIR/Healthedata1-Sandbox/input/examples-yaml/QuestionnaireResponse-AHC-HRSN-example.yml'
#survey_path = r'/Users/ehaas/Documents/FHIR/Healthedata1-Sandbox/input/examples-yaml/QuestionnaireResponse-PRAPARE-example.yml'
#survey_path = r'/Users/ehaas/Documents/FHIR/Healthedata1-Sandbox/input/examples-yaml/QuestionnaireResponse-hunger-vital-sign-example.yml'
#out_dir = Path.cwd() / 'output'
out_dir = r'/Users/ehaas/Documents/FHIR/Healthedata1-Sandbox/input/examples-yaml/'
path = Path(survey_path)

out_dir

'/Users/ehaas/Documents/FHIR/Healthedata1-Sandbox/input/examples-yaml/'

### Fetch Category and add to category

In [51]:
def get_category(category):
    cs_obj = CS.parse_file(category_path)
    for cat in cs_obj.concept:
        if cat.code == category:
            return {'coding': [{'system': cs_obj.url, 'code': cat.code, 'display': cat.display}]}

get_category('sdoh')   

{'coding': [{'system': 'http://hl7.org/fhir/us/core/CodeSystem/us-core-category',
   'code': 'sdoh',
   'display': 'SDOH'}]}

### Fetch Panels Observations and add hasMembers

In [52]:
def update_members(parent, members=[]):
    try:
        parent = parent.replace("/", "")
    except AttributeError:
        pass
    in_path = Path(out_dir) / f'Observation-{survey_name}-panel-example-{parent}.yml' #Path().cwd() / 'output' / f'Observation-{survey_name}-panel-example-{parent}.yml'
    my_obj = Obs.parse_file(in_path)
    # print(my_obj.yaml(indent=True))
    my_obj.hasMember = []
    for m in members:
        new_member = dict(reference = f'Observation/{m}')
        my_obj.hasMember.append(new_member)
    # print('='*80)
    # print(my_obj.yaml(indent=True))
    # print('='*80)
    write_out(my_obj)

### Write Observations to file

In [53]:
def write_out(fhir_obj):
    out_path = Path(out_dir) / f'Observation-{fhir_obj.id}.yml'
    # note need to the date time format is not json serializabel out of the box see QR-OBs-DE/YAML-JSON-date-serialization.ipynb
    out_path.write_text(fhir_obj.yaml())

### Fetch QuestionnaireResponse Instance from file

In [54]:
from fhir.resources.questionnaireresponse import QuestionnaireResponse as QR
qr_obj = QR.parse_file(path)
qr_obj.authored.isoformat()

'2022-11-29T20:50:32.718000+00:00'

### Create Observation Instance with the metadata

In [55]:
def get_coding(my_code):
    coding_obj = construct_fhir_element('Coding',my_code.valueCoding)
    return coding_obj

def get_ccode(my_coding):
    ccode_obj = construct_fhir_element('CodeableConcept',{})
    
    ccode_obj.coding = [my_coding]
    ccode_obj.text = my_coding.display

    return(ccode_obj)

def get_answers(obs_obj, answer):
    if answer.valueCoding:
      # print(answer.valueCoding)  
      obs_obj.valueCodeableConcept = get_ccode(answer.valueCoding)
    elif answer.valueString:
      obs_obj.valueString = answer.valueString 
    elif answer.valueDecimal:
      obs_obj.valueString  = answer.valueDecimal 
    #print(obs_obj.json(indent=True))
    return (obs_obj)


def create_obs (obs_id, text, obs_type='item', answers = None):
    obs_obj = Obs.parse_file("survey_obs.yml")
    print(f'{survey_name}-{obs_type}-example-{obs_id}')
    obs_obj.id = f'{survey_name}-{obs_type}-example-{obs_id}'
    obs_obj.meta.extension[0].valueString = f'{survey_name} {obs_type} Example {obs_id}'.title()
    obs_obj.meta.extension[1].valueMarkdown = f'This is a {obs_obj.meta.extension[0].valueString} ({text}) \
for the *US Core Observation Screening Assessment Profile*'
    obs_obj.meta.profile = [my_profile]
    cat2 = get_category(category)
    if cat2:
      obs_obj.category.append(cat2)
    # obs_obj.code.coding[0].system = 'http://loinc.org'
    obs_obj.code.coding[0].code = obs_id
    obs_obj.code.coding[0].display = text
    obs_obj.code.text = text
    if answers:
        get_answers(obs_obj, answers)
        obs_obj.hasMember = None
    obs_obj.effectiveDateTime = qr_obj.authored.isoformat()
    obs_obj.subject.reference = qr_obj.subject.reference
    obs_obj.performer[0].reference = qr_obj.author.reference
    obs_obj.derivedFrom[0].reference = f'QuestionnaireResponse/{qr_obj.id}'
    try:
      obs_obj.derivedFrom[0].display = qr_obj.meta.extension[0].valueString
    except TypeError:
      obs_obj.derivedFrom[0].display = None
    return(obs_obj)

### Use recursion to parse out the nested responses


- Create Observations only from those with values

In [56]:
def get_items(my_item, parent = None):

    try:
      assert my_item.item
    except (AttributeError, AssertionError):
        return  # item observations !!

    else: # groups observations !!

      my_list = []
      for i in my_item.item:  # <<< create new Observation here
        new_item = construct_fhir_element('QuestionnaireResponseItem', i)
        print(new_item.linkId)
        print(new_item.text)
        # get_answers(new_item.answer)
        try:
          assert new_item.answer
        except AssertionError:
            print ('NO Answer?  - group?\n\n')
            survey_obs_type = "panel"
            new_code = new_item.linkId.split("/")[-1]
            my_panel_obs = create_obs(new_code, new_item.text, survey_obs_type)
            my_list.append(my_panel_obs.id)
            write_out(my_panel_obs)

        else:
          print ('Answer  - item!\n\n')
          if len(new_item.answer) > 1:
            survey_obs_type = "multiselect-item"
            for i, answer in enumerate(new_item.answer):
                new_code = f'{new_item.linkId.split("/")[-1]}-answer{i}'
                my_item_obs = create_obs(new_code, new_item.text, survey_obs_type, answer)
                my_list.append(my_item_obs.id)
                write_out(my_item_obs)
          else:
            survey_obs_type = "item"
            new_code = new_item.linkId.split("/")[-1]
            my_item_obs = create_obs(new_code, new_item.text, survey_obs_type, new_item.answer[0])
            my_list.append(my_item_obs.id)
            write_out(my_item_obs)
          
        get_items(i, new_item.linkId)
        
        
      print(f'my_list for {parent} = {my_list}\n\n')
      try:
        update_members(parent, my_list)
      except Exception as e:
        print(e)
        if create_survey_panel:
            my_panel_obs = create_obs(survey_code, survey_code_name, "panel")
            write_out(my_panel_obs)
            update_members(survey_code, my_list)

get_items(qr_obj)




/44250-9
Little interest or pleasure in doing things in last 2 weeks
Answer  - item!


PHQ9-item-example-44250-9
/44255-8
Feeling down, depressed, or hopeless in last 2 weeks
Answer  - item!


PHQ9-item-example-44255-8
/44259-0
Trouble falling or staying asleep, or sleeping too much
Answer  - item!


PHQ9-item-example-44259-0
/44254-1
Feeling tired or having little energy
Answer  - item!


PHQ9-item-example-44254-1
/44251-7
Poor appetite or overeating
Answer  - item!


PHQ9-item-example-44251-7
/44258-2
Feeling bad about yourself-or that you are a failure or have let yourself or your family down
Answer  - item!


PHQ9-item-example-44258-2
/44252-5
Trouble concentrating on things, such as reading the newspaper or watching television
Answer  - item!


PHQ9-item-example-44252-5
/44253-3
Moving or speaking so slowly that other people could have noticed. Or the opposite-being so fidgety or restless that you have been moving around a lot more than usual
Answer  - item!


PHQ9-item-example-44